# 建立SQL的字典測試(class Sqlstructure)

In [2]:
from sqlparsemodel.Sqlstructure import Sqlstructure
from sqlparsemodel.Querystate import Querystate
import re 
Querystate = Querystate()

In [2]:
the_query = '''CREATE TABLE WORK.FINAL as 
                   select a1, count(a2), a3 
                   from AAA_table
                        left join (SELECT * FROM (SELECT * FROM MEOW.CCC_table ) as t1 ) as t2 on (t3.a1 = t2.a5)
                   where t3.X1 = 'thing' and t3.X2 in (SELECT distinct(b1) FROM MEOW.BBB_table ) 
                   Group by t3.a1, t3.a3
                   order by t3.a1, t3.a3
                   limit 15'''
the_query = '''CREATE TABLE DD01 AS
SELECT POLICY_NO,
    PROD_ID,
    INSD_ID_SAS,
    理賠金代號,
       CASE WHEN 初次殘疾鑑定日<>. THEN 初次殘疾鑑定日
   WHEN 初次殘疾鑑定日=. THEN 事故日
    END AS 初次殘疾鑑定日 FORMAT=YYMMDD10.
FROM (
 SELECT  DISTINCT 
   POLICY_NO,
   PROD_ID,
   INSD_ID_SAS,
   理賠金代號,
   min(初次殘疾鑑定日) FORMAT=YYMMDD10. AS 初次殘疾鑑定日,
   min(事故日) FORMAT=YYMMDD10. AS 事故日
   FROM(
    SELECT c.POLICY_NO,
        c.PROD_ID,
        b.理賠金代號,
        a.INSD_ID_SAS,
        c.DISB_DATE  FORMAT=YYMMDD10. AS 初次殘疾鑑定日,
        c.OCR_DATE  FORMAT=YYMMDD10. AS 事故日
    FROM QUERY.DTAAB001_Q as c
    INNER JOIN  b ON b.理賠金代號=c.CLAM_AMT_CODE AND c.PROD_ID=b.PROD_ID
    INNER JOIN a  ON c.OCR_ID_SAS=a.INSD_ID_SAS AND c.POLICY_NO=a.POLICY_NO
                 AND c.PROD_ID=a.PROD_ID
    WHERE c.PAY_STS not in ('0','5','6')
    )
 GROUP BY 1,2,3,4
 HAVING min(初次殘疾鑑定日) > 100 and min(事故日) < 2000
 )
WHERE 初次殘疾鑑定日<>. OR 事故日<>.'''

the_query = '''select * from aa
     union all
     select * from bb;'''

sql_parsed = Sqlstructure(the_query)



{}
|- 0 DML 'select'
|- 1 Whitespace ' '
|- 2 Wildcard '*'
|- 3 Whitespace ' '
|- 4 Keyword 'from'
|- 5 Whitespace ' '
|- 6 Identifier 'aa'
|  `- 0 Name 'aa'
|- 7 Newline ' '
|- 8 Whitespace ' '
|- 9 Whitespace ' '
|- 10 Whitespace ' '
|- 11 Whitespace ' '
|- 12 Whitespace ' '
|- 13 Keyword 'union ...'
|- 14 Newline ' '
|- 15 Whitespace ' '
|- 16 Whitespace ' '
|- 17 Whitespace ' '
|- 18 Whitespace ' '
|- 19 Whitespace ' '
|- 20 DML 'select'
|- 21 Whitespace ' '
|- 22 Wildcard '*'
|- 23 Whitespace ' '
|- 24 Keyword 'from'
|- 25 Whitespace ' '
|- 26 Identifier 'bb'
|  `- 0 Name 'bb'
`- 27 Punctuation ';'
None
SELECT表示句:SELECT
FROM表示句:FROM
SELECT表示句:SELECT
FROM表示句:FROM
1
SELECT表示句:SELECT
FROM表示句:FROM
SELECT表示句:SELECT
FROM表示句:FROM


In [3]:
sql_parsed.structured_dict

{'MAIN': {'SELECT': {'value': '* *'},
  'FROM': {'value': 'aa union all bb ;',
   'token': [[None, None, 'aa'], [None, None, 'bb']]}}}

In [50]:
query = '''


PROC SQL;
   CREATE TABLE MEOW.PAST_CLAIM_APLY_NO AS 
   SELECT DISTINCT t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
          t2.APLY_NO
      FROM Meow.POL_ALL_X_01
           INNER JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)
      WHERE t1.CREATE_DATE > t2.OCR_DATE ;
QUIT;



PROC SQL;
   CREATE TABLE WORK.PAST_CLAIM_ICDS AS 
   SELECT DISTINCT 
          t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
          t2.APLY_NO, 
          t2.DIAG_SER_NO, 
          t2.CFM_ICD_CODE1, 
          t2.CFM_ICD_CODE2, 
          t2.CFM_ICD_CODE3, 
          t2.CFM_ICD_CODE4, 
          t2.CFM_ICD_CODE5
      FROM MEOW.PAST_CLAIM_APLY_NO t1
	       INNER JOIN QUERY.DTAAA020_AE t2 ON (t1.APLY_NO = t2.APLY_NO);
QUIT;'''

# Split SAS PROC SQLs

In [51]:
sql_parsed_list = {}
query_id = 0
for string in re.split("PROC SQL;", query):
    if string.strip() == "":
        continue
    query_id += 1
    print("=======================")
    print(re.sub("QUIT;", "", string).strip())
    sql_parsed_list[ str(query_id) ] = Sqlstructure(string)

CREATE TABLE MEOW.PAST_CLAIM_APLY_NO AS 
   SELECT DISTINCT t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
          t2.APLY_NO
      FROM Meow.POL_ALL_X_01
           INNER JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)
      WHERE t1.CREATE_DATE > t2.OCR_DATE ;
{}
|- 0 Newline ' '
|- 1 Whitespace ' '
|- 2 Whitespace ' '
|- 3 Whitespace ' '
|- 4 DDL 'CREATE'
|- 5 Whitespace ' '
|- 6 Keyword 'TABLE'
|- 7 Whitespace ' '
|- 8 Identifier 'MEOW.P...'
|  |- 0 Name 'MEOW'
|  |- 1 Punctuation '.'
|  `- 2 Name 'PAST_C...'
|- 9 Whitespace ' '
|- 10 Keyword 'AS'
|- 11 Whitespace ' '
|- 12 Newline ' '
|- 13 Whitespace ' '
|- 14 Whitespace ' '
|- 15 Whitespace ' '
|- 16 DML 'SELECT'
|- 17 Whitespace ' '
|- 18 Keyword 'DISTIN...'
|- 19 Whitespace ' '
|- 20 IdentifierList 't1.CON...'
|  |- 0 Identifier 't1.CON...'
|  |  |- 0 Name 't1'
|  |  |- 1 Punctuation '.'
|  |  `- 2 Name 'CONTRA...'
|  |- 1 Punctuation ','
|  |- 2 Newline ' '
|  |- 3 Whitespace ' '
|  |- 4 Whitespace ' '
|  |- 5 White

## From to ID check and replace 
- 先放from_id : from_id == "" and to_id == "" 把now_id放到from_id
- 再放to_id : from_id != "" and to_id == "" 把now_id放到to_id
- id置換 : from_id != "" and to_id != "" to_id 取代 from_id，並把now_id放到_to_id

In [52]:
def from_to_check(from_id, to_id, now_id):
    if from_id == "" and to_id == "":
        from_id = now_id
        
    elif from_id != "" and to_id == "":
        to_id = now_id
        
    elif from_id != "" and to_id != "":
        from_id = to_id
        to_id = now_id
    
    return(from_id, to_id)

# PROC SQLs to relation structure
## Setting dataframes and dictinary
- NODE ID dataframe
- NODE_ID dict
- From to dataframe(NODE) 
- From Create dataframe (tables NODE)


### NODE 生成的方法 
- 第一階(Split query) ： query1, query2, ····
- 第二階(Subquery) ： query1_main, query1_main_from1 ···· , query2_main, 
  ····
- 第三階(State) ：FROM、WHERE、GROUPBY 
  ···· 
- 第四階(Value, token) 與第三階一起 ：FROM.TOKENS(第五階)、FROM.VALUE、WHERE.VALUE、
  ···· 

In [54]:
node_id = 0
node_dict = {}
node_property = [["node_id", # 0
                 "query_id", # 1
                 "subq_name", # 2
                 "state", # 3
                 "token_tag", # 4
                 "state_value", # 5
                 "state_alias", # 6
                 "state_realname", # 7
                 "state_parentname" # 8
                ]]

from_to_property = [["from", "to"]]

for item in sql_parsed_list.items():
    # 00 Setting query_id, query_dict
    ## 00_1 initial from_id and to_id To initial th relation
    query_id = item[0]
    subquery_dict = item[1].structured_dict
    from_id = ""
    to_id = ""
    
    # 01 Getting dict 
    ## 01_1 MAIN, MAIN_SELECT.....
    ## 01_2 "FROM", "WHERE", "GROUP BY", "HAVING", "SELECT", "ORDER BY", "LIMIT", "TABLE"
    ## 01_3 "token" get
    for item_subq in subquery_dict.items() : 
        subq_name = item_subq[0]
        subq_value = item_subq[1]
        ## 01_2
        for state in Querystate.nodeorder_statelist : 
            
            ## 01_2_0========================
            ### Check State In subquery_dict, if not continue
            if state not in subq_value.keys():
                continue
                
            ## 01_2_1=============================
            ### Record Node property
            ### 1. TABLE node has tokens, using only token node 
            ### 2. From node has token and value need to be different NODE
            node_id += 1
            if state == "TABLE":
                node_record = [ str(node_id), # 0
                              query_id, # 1
                              subq_name, # 2
                              state, # 3
                              "token", # 4
                              subquery_dict[subq_name][state]["value"], # 5
                              subquery_dict[subq_name][state]["token"][0][0], # 6
                              subquery_dict[subq_name][state]["token"][0][1], # 7
                              subquery_dict[subq_name][state]["token"][0][2] # 8
                              ]
            else:
                node_record = [ str(node_id), # 0
                              query_id, # 1
                              subq_name, # 2
                              state, # 3
                              "value", # 4
                              subquery_dict[subq_name][state]["value"], # 5
                              "", # 6
                              "", # 7
                              "" # 8
                              ]
            node_property.append( node_record )
            node_dict[ str(node_id) ] = node_record
            
            
            ## 01_2_2=============================
            ### Record relation (from to)
            from_id, to_id = from_to_check(from_id = from_id, to_id = to_id, now_id = node_id)
            
            if from_id != "" and to_id != "" :
                from_to_property.append([from_id, to_id])
            
            
            ## 01_3 =================================
            ### Token gets 
            ### Get value id be a to_id
            if state == "FROM" :
                to_id = from_id
                for from_tb in subq_value[state]["token"] :
                    node_id += 1
                    node_record = [ str(node_id), # 0
                              query_id, # 1
                              subq_name, # 2
                              state, # 3
                              "token", # 4
                              ",".join(filter(None, from_tb)), # 5
                              from_tb[0], # 6
                              from_tb[1], # 7
                              from_tb[2] # 8
                              ]
                    node_property.append( node_record )
                    node_dict[ str(node_id) ] = node_record
                    
                    #### Setting id 
                    from_id = node_id
                    if from_id != "" and to_id != "" :
                        from_to_property.append([from_id, to_id])
                    
#             
            
    

# Functions : sassql_structure 

## From to ID check and replace 
- 先放from_id : from_id == "" and to_id == "" 把now_id放到from_id
- 再放to_id : from_id != "" and to_id == "" 把now_id放到to_id
- id置換 : from_id != "" and to_id != "" to_id 取代 from_id，並把now_id放到_to_id

In [60]:
def from_to_check(from_id, to_id, now_id):
    if from_id == "" and to_id == "":
        from_id = now_id
        
    elif from_id != "" and to_id == "":
        to_id = now_id
        
    elif from_id != "" and to_id != "":
        from_id = to_id
        to_id = now_id
    
    return(from_id, to_id)

In [99]:
def none_to_empty(string):
    if string is None :
        return("")
    else:
        return(string)

In [61]:
def sassql_parsed(query):
    sql_parsed_list = {}
    query_id = 0
    for string in re.split("PROC SQL;", query):
        if string.strip() == "":
            continue
        query_id += 1
        print("=======================")
        print(re.sub("QUIT;", "", string).strip())
        sql_parsed_list[ str(query_id) ] = Sqlstructure(string)
        
    return( sql_parsed_list )

## PROC SQLs to relation structure
### Setting dataframes and dictinary
- NODE ID dataframe
- NODE_ID dict
- From to dataframe(NODE) 
- From Create dataframe (tables NODE)


### NODE 生成的方法 
- 第一階(Split query) ： query1, query2, ····
- 第二階(Subquery) ： query1_main, query1_main_from1 ···· , query2_main, 
  ····
- 第三階(State) ：FROM、WHERE、GROUPBY 
  ···· 
- (Value, token) 與第三階一起 ：Value 記錄一個node 、 tokens各別記錄一個node
  ···· 

In [158]:
def sassql_nodelist(sql_parsed_list):
    node_id = 0
    node_dict = {}
    node_property = [["node_id", # 0
                     "query_id", # 1
                     "subq_name", # 2
                     "state", # 3
                     "token_tag", # 4
                     "state_value", # 5
                     "state_alias", # 6
                     "state_parentname", # 7
                     "state_realname" # 8
                    ]]

    from_to_property = [["from", "to"]]

    for item in sql_parsed_list.items():
        ## initial var : Setting query_id, subquery_dict 
        ## 00_1 initial from_id and to_id To initial th relation by Different query
        query_id = item[0]
        subquery_dict = item[1].structured_dict
        from_id = ""
        to_id = ""

        # 01 Getting dict 
        ## 01_1 MAIN, MAIN_SELECT.....
        ## 01_2 "FROM", "WHERE", "GROUP BY", "HAVING", "SELECT", "ORDER BY", "LIMIT", "TABLE"
        ## 01_3 "token" get
        for item_subq in subquery_dict.items() : ## 01_1
            
            ## initial var : Get subquery name and Dict
            subq_name = item_subq[0]
            subq_value = item_subq[1]
            for state in Querystate.nodeorder_statelist : ## 01_2
                
                ## initial var : one node property  
                node_record = []
                
                ## 01_2_0========================
                ### SET Subquery (MAIN_...) TABLE to the subname
                ### Check State In subquery_dict, if not continue
                if re.search("MAIN_", subq_name) and state == "TABLE":
                    subq_value[ "TABLE" ] = {"value" : "SUBQ." + subq_name, 
                                             "token" : [["", "SUBQ", subq_name]] }
                if state not in subq_value.keys():
                    continue

                ## 01_2_1=============================
                ### Record Node property
                ### 1. TABLE node has tokens, using only token node 
                ### 2. From node has token and value need to be different NODE
                node_id += 1
                if state == "TABLE":
                    node_record = [ str(node_id), # 0
                                  query_id, # 1
                                  subq_name, # 2
                                  state, # 3
                                  "token", # 4
                                  subq_value[state]["value"], # 5
                                  none_to_empty( subq_value[state]["token"][0][0] ), # 6
                                  none_to_empty( subq_value[state]["token"][0][1] ), # 7
                                  none_to_empty( subq_value[state]["token"][0][2] ) # 8
                                  ]
                else:
                    node_record = [ str(node_id), # 0
                                  query_id, # 1
                                  subq_name, # 2
                                  state, # 3
                                  "value", # 4
                                  subquery_dict[subq_name][state]["value"], # 5
                                  "", # 6
                                  "", # 7
                                  "" # 8
                                  ]
                node_property.append( node_record )
                node_dict[ str(node_id) ] = node_record


                ## 01_2_2=============================
                ### Record relation (from to)
                from_id, to_id = from_to_check(from_id = from_id, to_id = to_id, now_id = node_id)

                if from_id != "" and to_id != "" :
                    from_to_property.append([str(from_id), str(to_id)])


                ## 01_3 =================================
                ### Token gets 
                ### Get value id be a to_id
                if state == "FROM" :
                    to_id = from_id
                    for from_tb in subquery_dict[subq_name][state]["token"] :
                        node_id += 1
                        node_record = [ str(node_id), # 0
                                  query_id, # 1
                                  subq_name, # 2
                                  state, # 3
                                  "token", # 4
                                  "alias:" + none_to_empty( from_tb[0] ) + ", " + "Database:" + none_to_empty( from_tb[1] ) + ", " +  "Table:" + none_to_empty( from_tb[2] ), # 5
                                  none_to_empty( from_tb[0] ), # 6
                                  none_to_empty( from_tb[1] ), # 7
                                  none_to_empty( from_tb[2] )# 8
                                  ]
                        node_property.append( node_record )
                        node_dict[ str(node_id) ] = node_record

                        #### Record relation (from to)
                        from_id = node_id
                        if from_id != "" and to_id != "" :
                            from_to_property.append([str(from_id), str(to_id)])
    return(node_dict, node_property, from_to_property)

## Query relation (NODE table相連 (state FROM and TABLE))

In [199]:
import pandas as pd 
node_df = pd.DataFrame(node_property[1:], columns = node_property[0])
fromtokens = node_df[ (node_df["state"] == "FROM") & (node_df["token_tag"] == "token") & (node_df["state_parentname"] != "SUBQ") ]
tabletokens = node_df[ (node_df["state"] == "TABLE") & (node_df["token_tag"] == "token") & (node_df["state_parentname"] != "SUBQ")]
sub_fromtokens = node_df[ (node_df["state"] == "FROM") & (node_df["token_tag"] == "token") & (node_df["state_parentname"] == "SUBQ") ]
sub_tabletokens = node_df[ (node_df["state"] == "TABLE") & (node_df["token_tag"] == "token") & (node_df["state_parentname"] == "SUBQ") ]


relation = pd.merge(tabletokens, fromtokens, on = ["state_parentname", "state_realname"], how = "inner")
sub_relation = pd.merge(sub_tabletokens, sub_fromtokens, on = ["state_parentname", "state_realname"], how = "inner")

from_to_query = [['from', 'to']]
from_to_query = from_to_query + relation[["node_id_x", "node_id_y"]].values.tolist()
from_to_query = from_to_query + sub_relation[["node_id_x", "node_id_y"]].values.tolist()

In [200]:
from_to_query

[['from', 'to'], ['6', '8'], ['15', '9']]

In [178]:
relation

pandas.core.frame.DataFrame

In [139]:
table_tokens

,node_id,query_id,subq_name,state,token_tag,state_value,state_alias,state_realname,state_parentname
5,6,1,MAIN,TABLE,token,MEOW.PAST_CLAIM_APLY_NO,,MEOW,PAST_CLAIM_APLY_NO
10,11,2,MAIN,TABLE,token,WORK.PAST_CLAIM_ICDS,,WORK,PAST_CLAIM_ICDS


In [186]:
query = '''


PROC SQL;
   CREATE TABLE MEOW.PAST_CLAIM_APLY_NO AS 
   SELECT DISTINCT t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
          t2.APLY_NO
      FROM Meow.POL_ALL_X_01
           INNER JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)
      WHERE t1.CREATE_DATE > t2.OCR_DATE ;
QUIT;



PROC SQL;
   CREATE TABLE WORK.PAST_CLAIM_ICDS AS 
   SELECT DISTINCT 
          t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
          t2.APLY_NO, 
          t2.DIAG_SER_NO, 
          t2.CFM_ICD_CODE1, 
          t2.CFM_ICD_CODE2, 
          t2.CFM_ICD_CODE3, 
          t2.CFM_ICD_CODE4, 
          t2.CFM_ICD_CODE5
      FROM MEOW.PAST_CLAIM_APLY_NO t1
	       INNER JOIN QUERY.DTAAA020_AE t2 ON (t1.APLY_NO = t2.APLY_NO);
QUIT;'''

query = '''


PROC SQL;
   CREATE TABLE MEOW.PAST_CLAIM_APLY_NO AS 
   SELECT DISTINCT t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
          t2.APLY_NO
      FROM Meow.POL_ALL_X_01
           INNER JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)
      WHERE t1.CREATE_DATE > t2.OCR_DATE ;
QUIT;



PROC SQL;
   CREATE TABLE WORK.PAST_CLAIM_ICDS AS 
   SELECT DISTINCT 
          t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
          t2.APLY_NO, 
          t2.DIAG_SER_NO, 
          t2.CFM_ICD_CODE1, 
          t2.CFM_ICD_CODE2, 
          t2.CFM_ICD_CODE3, 
          t2.CFM_ICD_CODE4, 
          t2.CFM_ICD_CODE5
      FROM MEOW.PAST_CLAIM_APLY_NO t1
	       INNER JOIN (SELECT * FROM QUERY.DTAAA020_AE) t2 ON (t1.APLY_NO = t2.APLY_NO);
QUIT;'''

parsed_list = sassql_parsed(query)
node_dict, node_property, from_to_property = sassql_nodelist(parsed_list)

CREATE TABLE MEOW.PAST_CLAIM_APLY_NO AS 
   SELECT DISTINCT t1.CONTRACT_NO,
          t1.INSD_ID_SAS,
          t2.APLY_NO
      FROM Meow.POL_ALL_X_01
           INNER JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)
      WHERE t1.CREATE_DATE > t2.OCR_DATE ;
{}
|- 0 Newline ' '
|- 1 Whitespace ' '
|- 2 Whitespace ' '
|- 3 Whitespace ' '
|- 4 DDL 'CREATE'
|- 5 Whitespace ' '
|- 6 Keyword 'TABLE'
|- 7 Whitespace ' '
|- 8 Identifier 'MEOW.P...'
|  |- 0 Name 'MEOW'
|  |- 1 Punctuation '.'
|  `- 2 Name 'PAST_C...'
|- 9 Whitespace ' '
|- 10 Keyword 'AS'
|- 11 Whitespace ' '
|- 12 Newline ' '
|- 13 Whitespace ' '
|- 14 Whitespace ' '
|- 15 Whitespace ' '
|- 16 DML 'SELECT'
|- 17 Whitespace ' '
|- 18 Keyword 'DISTIN...'
|- 19 Whitespace ' '
|- 20 IdentifierList 't1.CON...'
|  |- 0 Identifier 't1.CON...'
|  |  |- 0 Name 't1'
|  |  |- 1 Punctuation '.'
|  |  `- 2 Name 'CONTRA...'
|  |- 1 Punctuation ','
|  |- 2 Newline ' '
|  |- 3 Whitespace ' '
|  |- 4 Whitespace ' '
|  |- 5 White

In [96]:
node_dict

{'1': ['1',
  '1',
  'MAIN',
  'FROM',
  'value',
  'Meow.POL_ALL_X_01 INNER JOIN QUERY.DTAAB001_Q t2 ON (t1.INSD_ID_SAS = t2.OCR_ID_SAS)',
  '',
  '',
  ''],
 '2': ['2',
  '1',
  'MAIN',
  'FROM',
  'token',
  'Meow,POL_ALL_X_01',
  '',
  'Meow',
  'POL_ALL_X_01'],
 '3': ['3',
  '1',
  'MAIN',
  'FROM',
  'token',
  't2,QUERY,DTAAB001_Q',
  't2',
  'QUERY',
  'DTAAB001_Q'],
 '4': ['4',
  '1',
  'MAIN',
  'WHERE',
  'value',
  'WHERE t1.CREATE_DATE > t2.OCR_DATE ;',
  '',
  '',
  ''],
 '5': ['5',
  '1',
  'MAIN',
  'SELECT',
  'value',
  'DISTINCT t1.CONTRACT_NO,t1.INSD_ID_SAS,t2.APLY_NO',
  '',
  '',
  ''],
 '6': ['6',
  '1',
  'MAIN',
  'TABLE',
  'token',
  'MEOW.PAST_CLAIM_APLY_NO',
  '',
  'MEOW',
  'PAST_CLAIM_APLY_NO'],
 '7': ['7',
  '2',
  'MAIN',
  'FROM',
  'value',
  'MEOW.PAST_CLAIM_APLY_NO t1 INNER JOIN QUERY.DTAAA020_AE t2 ON (t1.APLY_NO = t2.APLY_NO) ;',
  '',
  '',
  ''],
 '8': ['8',
  '2',
  'MAIN',
  'FROM',
  'token',
  't1,MEOW,PAST_CLAIM_APLY_NO',
  't1',
  'MEOW',

In [191]:
pd.DataFrame(node_property)

,0,1,2,3,4,5,6,7,8
0,node_id,query_id,subq_name,state,token_tag,state_value,state_alias,state_parentname,state_realname
1,1,1,MAIN,FROM,value,Meow.POL_ALL_X_01 INNER JOIN QUERY.DTAAB001_Q ...,,,
2,2,1,MAIN,FROM,token,"alias:, Database:Meow, Table:POL_ALL_X_01",,Meow,POL_ALL_X_01
3,3,1,MAIN,FROM,token,"alias:t2, Database:QUERY, Table:DTAAB001_Q",t2,QUERY,DTAAB001_Q
4,4,1,MAIN,WHERE,value,WHERE t1.CREATE_DATE > t2.OCR_DATE ;,,,
5,5,1,MAIN,SELECT,value,"DISTINCT t1.CONTRACT_NO,t1.INSD_ID_SAS,t2.APLY_NO",,,
6,6,1,MAIN,TABLE,token,MEOW.PAST_CLAIM_APLY_NO,,MEOW,PAST_CLAIM_APLY_NO
7,7,2,MAIN,FROM,value,MEOW.PAST_CLAIM_APLY_NO t1 INNER JOIN SUBQ.MAI...,,,
8,8,2,MAIN,FROM,token,"alias:t1, Database:MEOW, Table:PAST_CLAIM_APLY_NO",t1,MEOW,PAST_CLAIM_APLY_NO
9,9,2,MAIN,FROM,token,"alias:t2, Database:SUBQ, Table:MAIN_FROM1",t2,SUBQ,MAIN_FROM1


In [95]:
from_to_property

[['from', 'to'],
 ['2', '1'],
 ['3', '1'],
 ['1', '4'],
 ['4', '5'],
 ['5', '6'],
 ['8', '7'],
 ['9', '7'],
 ['7', '10'],
 ['10', '11']]

In [78]:
subquery_dict

{'MAIN': {'': {'value': 'CREATE'},
  'TABLE': {'value': 'WORK.PAST_CLAIM_ICDS',
   'token': [[None, 'WORK', 'PAST_CLAIM_ICDS']]},
  'OTHER': {'value': 'AS'},
  'SELECT': {'value': 'DISTINCT t1.CONTRACT_NO,t1.INSD_ID_SAS,t2.APLY_NO, t2.DIAG_SER_NO, t2.CFM_ICD_CODE1, t2.CFM_ICD_CODE2, t2.CFM_ICD_CODE3, t2.CFM_ICD_CODE4, t2.CFM_ICD_CODE5'},
  'FROM': {'value': 'MEOW.PAST_CLAIM_APLY_NO t1 INNER JOIN QUERY.DTAAA020_AE t2 ON (t1.APLY_NO = t2.APLY_NO) ;',
   'token': [['t1', 'MEOW', 'PAST_CLAIM_APLY_NO'],
    ['t2', 'QUERY', 'DTAAA020_AE']]}}}

In [34]:
subquery_dict

{'MAIN': {'': {'value': 'CREATE'},
  'TABLE': {'value': 'WORK.PAST_CLAIM_ICDS',
   'token': [[None, 'WORK', 'PAST_CLAIM_ICDS']]},
  'OTHER': {'value': 'AS'},
  'SELECT': {'value': 'DISTINCT t1.CONTRACT_NO,t1.INSD_ID_SAS,t2.APLY_NO, t2.DIAG_SER_NO, t2.CFM_ICD_CODE1, t2.CFM_ICD_CODE2, t2.CFM_ICD_CODE3, t2.CFM_ICD_CODE4, t2.CFM_ICD_CODE5'},
  'FROM': {'value': 'MEOW.PAST_CLAIM_APLY_NO t1 INNER JOIN QUERY.DTAAA020_AE t2 ON (t1.APLY_NO = t2.APLY_NO) ;',
   'token': [['t1', 'MEOW', 'PAST_CLAIM_APLY_NO'],
    ['t2', 'QUERY', 'DTAAA020_AE']]}}}

In [28]:
sql_parsed_list[2].structured_dict

{'MAIN': {'': {'value': 'CREATE'},
  'TABLE': {'value': 'WORK.PAST_CLAIM_ICDS',
   'token': [[None, 'WORK', 'PAST_CLAIM_ICDS']]},
  'OTHER': {'value': 'AS'},
  'SELECT': {'value': 'DISTINCT t1.CONTRACT_NO,t1.INSD_ID_SAS,t2.APLY_NO, t2.DIAG_SER_NO, t2.CFM_ICD_CODE1, t2.CFM_ICD_CODE2, t2.CFM_ICD_CODE3, t2.CFM_ICD_CODE4, t2.CFM_ICD_CODE5'},
  'FROM': {'value': 'MEOW.PAST_CLAIM_APLY_NO t1 INNER JOIN QUERY.DTAAA020_AE t2 ON (t1.APLY_NO = t2.APLY_NO) ;',
   'token': [['t1', 'MEOW', 'PAST_CLAIM_APLY_NO'],
    ['t2', 'QUERY', 'DTAAA020_AE']]}}}